In [ ]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import sys

In [ ]:
for config in ['eddy', 'jet']:
    for forcing in ['forcing1', 'forcing2', 'forcing3', 'high_res', 'low_res']:
        ds = xr.open_zarr(fsspec.get_mapper(f'/scratch/zanna/data/pyqg/publication/{config}/{forcing}.zarr'), consolidated=True)

In [ ]:
ds.attrs

In [ ]:
ds

In [ ]:
ds.plot.scatter(x="time",y="APEflux")

In [ ]:
plt.imshow(ds.q.isel(run=2,time=30,lev=1))

In [ ]:
sys.getsizeof(ds)